# Introducción a la Similitud de Imágenes

En este notebook, presentaremos la búsqueda de imágenes utilizando `sentence-transformers`, una biblioteca de Python para embeddings de frases, texto e imágenes de última generación.

<b>¿Qué es la Similitud de Imágenes?</b>

La similitud de imágenes se refiere al proceso de encontrar imágenes que sean visualmente parecidas. Esto puede ir desde encontrar duplicados casi idénticos hasta agrupar imágenes basadas en una semejanza temática. Las implicaciones de esta tecnología son profundas, ya que sustenta sistemas de:

* <b>Búsqueda Visual</b>: Los minoristas y mercados en línea utilizan la similitud de imágenes para ofrecer recomendaciones de productos basadas en fotos subidas por el usuario. Esta tecnología mejora la experiencia de compra al permitir a los usuarios buscar productos usando imágenes en lugar de palabras.

* <b>Descubrimiento de Contenido</b>: Las plataformas de redes sociales y los sistemas de gestión de contenidos confían en la similitud de imágenes para categorizar y recomendar contenido, ayudando a los usuarios a descubrir nuevas publicaciones relacionadas con lo que ya les gusta.

* <b>Archivado Digital</b>: En bibliotecas y archivos, la similitud de imágenes ayuda a organizar, indexar y recuperar contenido visual de vastas bases de datos, facilitando la búsqueda de documentos históricos y obras de arte.

* <b>Seguridad y Vigilancia</b>: Los algoritmos de similitud de imágenes pueden identificar objetos o personas de interés en diferentes fotogramas de vídeo o ubicaciones, contribuyendo a los esfuerzos de seguridad y aplicación de la ley.

* <b>Atención Sanitaria</b>: En la imagenología médica, las medidas de similitud pueden ayudar a identificar historiales de casos similares, comprender la progresión de enfermedades e incluso asistir en el diagnóstico mediante la comparación de escaneos de pacientes con una base de datos de condiciones conocidas.

<b>¿Por qué es un desafío?</b>

Las imágenes pueden variar en tamaño, ángulo, iluminación e incluso estar parcialmente ocultas. Los métodos tradicionales que dependen de coincidencias exactas no logran proporcionar resultados relevantes bajo estas condiciones. Por ello, las técnicas modernas de similitud de imágenes emplean el aprendizaje profundo *(deep learning)*, aprovechando particularmente modelos como CLIP *(Contrastive Language-Image Pretraining)* desarrollado por OpenAI, para entender y cuantificar la semejanza de una manera que imita la percepción humana.

<b>Objetivos de este Tutorial</b>

En este tutorial, profundizaremos en cómo el aprendizaje profundo, especialmente mediante el uso de Sentence Transformers y el modelo CLIP, nos permite mapear imágenes y textos en un espacio vectorial compartido. Este mapeo nos permite realizar tareas de búsqueda y recuperación matizadas, ofreciendo un puente entre las descripciones textuales y el contenido visual.

Al final de esta sesión, comprenderás cómo:

1. Utilizar el modelo CLIP para crear embeddings de imágenes y texto.

2. Implementar un sistema de búsqueda de imágenes que pueda encontrar imágenes similares basadas en consultas de texto.

3. Explorar aplicaciones prácticas y consideraciones al desplegar modelos de similitud de imágenes.

## Búsqueda de imágenes

En este notebook, presentaremos la búsqueda de imágenes utilizando Sentence Transformers, mapeando imágenes y textos en el mismo espacio vectorial. Esto nos permite realizar tareas de búsqueda y recuperación de imágenes basadas en descripciones textuales.

Para lograrlo, utilizaremos el modelo [CLIP (Contrastive Language-Image Pretraining)](https://openai.com/research/clip), el cual está diseñado para aprender un espacio de embedding (incrustación) conjunto tanto para imágenes como para textos.

CLIP es un modelo de IA desarrollado por OpenAI. Está diseñado para aprender de una amplia gama de tareas aprovechando la conexión entre el lenguaje natural y las imágenes.

* <b>Aprendizaje Multimodal</b>: CLIP es un modelo multimodal que puede entender tanto imágenes como texto. Está preentrenado en un conjunto de datos masivo que contiene pares de imágenes y sus respectivos subtítulos o descripciones de texto, aprendiendo a asociar conceptos visuales con el lenguaje natural.

* <b>Aprendizaje Contrastivo (Contrastive Learning)</b>: CLIP aprende optimizando un objetivo contrastivo. Se entrena para reconocer qué pares de imagen-subtítulo son correctos entre un conjunto de ejemplos negativos. Al aprender a puntuar los pares correctos de imagen-texto por encima de los incorrectos, el modelo aprende una representación útil para ambas modalidades.

* <b>Arquitectura</b>: CLIP utiliza una arquitectura basada en Transformers para procesar texto y una arquitectura Vision Transformer (ViT) o ResNet para procesar imágenes. Los codificadores (encoders) de imagen y texto se entrenan conjuntamente, lo que permite al modelo alinear ambas modalidades en un espacio de embedding compartido.

In [ ]:
# Instalación de la libreria sentence-transformers
# !pip install -U sentence-transformers

In [ ]:
# Importación de librerias
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from PIL import Image
import glob
import pickle
import zipfile
import copy
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm.autonotebook import tqdm

In [ ]:
# First, we load the respective CLIP model
model_name = 'clip-ViT-B-32' # Utilizamos el modelo mencionado
model = SentenceTransformer(model_name) # Lo cargamos

In [ ]:
import requests
from io import StringIO, BytesIO

def get_image_from_url(url): # Definimos función para descargar imagenes
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

Para buscar imágenes, necesitamos un conjunto de imágenes.

In [ ]:
img_url_path = 'https://github.com/ezponda/intro_deep_learning/raw/main/images/'
img_urls = [
    f'{img_url_path}eiffel_tower.jpeg',
    f'{img_url_path}taj_mahal.jpeg',
    f'{img_url_path}colosseum.jpeg',
    f'{img_url_path}great_wall_of_china.jpeg',
    f'{img_url_path}statue_of_liberty.jpeg',
] # Descargamos varias imagenes de internet, concretamente del git del profesor

images = [get_image_from_url(url) for url in img_urls]

print('Sample images: ')
for url, image in zip(img_urls, images):
    print('_'*50)
    print(f'url: {url}')
    display(image)

In [ ]:
img_embeddings = model.encode(images,
                       batch_size=128,
                       convert_to_tensor=True,
                       show_progress_bar=True) # Procesamos las imagenes y las pasamos a embeddings
img_embeddings = img_embeddings.cpu()
print(img_embeddings.shape)

Ahora, definamos una función para realizar la búsqueda de imágenes, dada una consulta y una lista de embeddings de imágenes.

In [ ]:
from typing import List, Union

def image_search(query: str, model: SentenceTransformer, img_embeddings: np.ndarray, # Función para encotnrar las imagenes
                 images: List[Image.Image], top_k: int = 2) -> None:
    """Realiza una búsqueda de imágenes dada una consulta de texto.

    Esta función calcula la similitud del coseno entre el embedding de la consulta y los
    embeddings de las imágenes, recupera las top_k imágenes con la mayor similitud y las muestra.

  Args:
        query (str): La consulta de búsqueda como una cadena de texto.
        model (SentenceTransformer): El modelo SentenceTransformer utilizado para codificar el texto y las imágenes.
        img_embeddings (np.ndarray): Embeddings precalculados de las imágenes.
        images (List[Image.Image]): Una lista de objetos de imagen de PIL.
        top_k (int): El número de mejores resultados a mostrar

    """ # Ecplicación de la función
    query_embedding = model.encode([query])[0]  # Transformarmos la query a un vector

    similarities = cosine_similarity([query_embedding], img_embeddings)[0] # computamos las similaridades

    top_k_indices = np.argsort(-similarities)[:top_k] # Indices de las imagenes mas parecidas

    print(f"Input query: {query}\n") # Printea el input de la query

    # Muestra as imagenes y las similaridades en función de la query
    for index in top_k_indices:
        print('_' * 50)
        print(f"Similarity Score: {similarities[index]:.4f}")  # Improved readability with formatting
        display(images[index])

In [ ]:
image_search('A building in Paris', model, img_embeddings, images, top_k=2) # Probamos el modelo

In [ ]:
image_search('Find me an image of a famous monument in India', model, img_embeddings, images, top_k=2)

In [ ]:
image_search('A building in China', model, img_embeddings, images, top_k=2)

## Dataset de subconjunto de Unsplash

[Unsplash](https://unsplash.com/data) es un conjunto de datos de imágenes colaborativo que se comparte de forma abierta.

*Vayamos a hacerlo con un dataset mayor*


In [ ]:
# Descargamos el dataset mayor, Unsplash
img_folder = './photos/'

if not os.path.exists(img_folder) or len(os.listdir(img_folder)) == 0:
    os.makedirs(img_folder, exist_ok=True)

    photo_filename = 'unsplash-25k-photos.zip'
    if not os.path.exists(photo_filename):
        util.http_get('http://sbert.net/datasets/'+photo_filename, photo_filename)

    #
    with zipfile.ZipFile(photo_filename, 'r') as zf:
        for member in tqdm(zf.infolist(), desc='Extracting'):
            zf.extract(member, img_folder)

In [ ]:
def read_image_from_path(file_path): # Definicion función para leer la imagen de un url
    img = Image.open(file_path) # Abre la imagen de file_path
    return img

use_precomputed_embeddings = True

if use_precomputed_embeddings:
    emb_filename = 'unsplash-25k-photos-embeddings.pkl'
    if not os.path.exists(emb_filename):
        util.http_get('http://sbert.net/datasets/'+emb_filename, emb_filename)

    with open(emb_filename, 'rb') as fIn:
        img_names, img_embeddings = pickle.load(fIn)


    print("Images:", len(img_names))
else:
    img_names = list(glob.glob('photos/*.jpg'))[:5_000]
    print("Images:", len(img_names))
    images = [read_image_from_path(img_name) for img_name in  img_names]
    img_embeddings = model.encode(images, batch_size=128, convert_to_tensor=True, show_progress_bar=True)
    img_embeddings = img_embeddings.cpu()

In [ ]:
from typing import List, Union
import os
from PIL import Image
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import copy

def image_search_from_path(query, model: SentenceTransformer, img_embeddings: np.ndarray,
                           img_folder: str, img_names: List[str], top_k: int = 2) -> None:
    """Aquí tienes la traducción al español, manteniendo el formato técnico de la documentación:

Realiza una búsqueda de imágenes para una consulta textual dada dentro de un conjunto de imágenes ubicadas en una carpeta específica.

   Args:
        query (str o Image.Image): La consulta textual para buscar imágenes similares.
        model (SentenceTransformer): El modelo SentenceTransformer utilizado para la codificación.
        img_embeddings (np.ndarray): Los embeddings precalculados de las imágenes.
        img_folder (str): La carpeta donde se encuentran almacenadas las imágenes.
        img_names (List[str]): Los nombres de archivo de las imágenes.
        top_k (int): El número de mejores resultados a devolver.
    """
    try:
        query_embedding = model.encode([query])[0]
        similarities = cosine_similarity([query_embedding], img_embeddings)[0]
        indexes = np.argpartition(similarities, -top_k)[-top_k:]
        indexes = indexes[np.argsort(-similarities[indexes])]

        print(f"Input query: {query}\n")
        for index in indexes:
            similarity_score = similarities[index]
            image_name = img_names[index]
            image_path = os.path.join(img_folder, image_name)
            try:
                with Image.open(image_path) as img:
                    print('_' * 50)
                    print(f"Similarity: {similarity_score:.4f}")
                    display(copy.deepcopy(img))
            except Exception as e:
                print(f"Error displaying image {image_name}: {e}")
    except Exception as e:
        print(f"Error in image search: {e}")


In [ ]:
image_search_from_path('A building in Paris', model, img_embeddings, img_folder, img_names, top_k=2)

In [ ]:
image_search_from_path('A building in China', model, img_embeddings, img_folder, img_names, top_k=2)

In [ ]:
image_search_from_path('Two dogs playing in the snow', model, img_embeddings, img_folder, img_names, top_k=2)

In [ ]:
image_search_from_path('Best food in the world', model, img_embeddings, img_folder, img_names, top_k=2)

## Búsqueda de imagen a imagen (Image-to-Image Search)

También puedes utilizar este método para realizar búsquedas de imagen a imagen. Para lograrlo, debes pasar `get_image_from_url(url)` al método de búsqueda. Este devolverá entonces imágenes similares.

In [ ]:
img = get_image_from_url(img_urls[0])
img

In [ ]:
image_search_from_path(img, model, img_embeddings, img_folder, img_names, top_k=5)